# Less features, random Forest

Same features in https://github.com/diogodutra/COVID-19-Albert_Eintein/blob/master/COVID-19_Albert_Einstein.ipynb


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
    mean_absolute_error
)
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_excel("E:/rohan/CSE 6/covid-19-master/covid-19-master/dataset.xlsx")

In [3]:
features = [
    'Leukocytes',
    'Monocytes',
    'Platelets',
    'Patient age quantile',
]

df_clean = df[features + ["SARS-Cov-2 exam result"]]

print(df_clean.shape)

# predicted label as simple integers
df_clean["SARS-Cov-2 exam result"].replace("positive", 1, inplace=True)
df_clean["SARS-Cov-2 exam result"].replace("negative", 0, inplace=True)

(5644, 5)


l:\Users\rohan\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [4]:
df_clean = df_clean.dropna()
print(df_clean.shape)

(601, 5)


In [5]:
X = df_clean.drop("SARS-Cov-2 exam result", axis=1)
y = df_clean["SARS-Cov-2 exam result"]

In [6]:
# scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(X)

l:\Users\rohan\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


array([[0.29434284, 0.37647056, 0.16839917, 0.89473684],
       [0.36452575, 0.33333333, 0.32952185, 0.05263158],
       [0.1735432 , 0.51372545, 0.17567568, 0.47368421],
       ...,
       [0.2245853 , 0.45882351, 0.17047818, 0.78947368],
       [0.04381117, 0.52941174, 0.06444907, 0.89473684],
       [0.11186731, 0.40784312, 0.13617464, 1.        ]])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(y_train.value_counts())
print(y_test.value_counts())

0    411
1     69
Name: SARS-Cov-2 exam result, dtype: int64
0    107
1     14
Name: SARS-Cov-2 exam result, dtype: int64


In [8]:
def _extract_feature_importance(model, features):
    if not hasattr(model, "coef_") and not hasattr(model, "feature_importances_"):
        raise Exception("Not possible to collect feature importances")

    if hasattr(model, "coef_"):
        model_feature_importances = model.coef_[0]
    elif hasattr(model, "feature_importances_"):
        model_feature_importances = model.feature_importances_

    return [
        (feature, importance)
        for feature, importance in sorted(
            zip(features, model_feature_importances),
            key=lambda pair: pair[1],
            reverse=True,
        )
    ]

def run_single(X_train, y_train, X_test, y_test, which_model):

    model = which_model["model"]
    
    model.fit(X_train, y_train)

    # let us just print the performance on the train set
    predictions_train = model.predict(X_train)
    train_results = {
        "prec": precision_score(y_train, predictions_train),
        "rec": recall_score(y_train, predictions_train),
        "roc": roc_auc_score(y_train, predictions_train),
    }
    cm_train = confusion_matrix(y_train, predictions_train)
    
    # in the test set
    predictions_test = model.predict(X_test)
    test_results = {
        "prec": precision_score(y_test, predictions_test),
        "rec": recall_score(y_test, predictions_test),
        "roc": roc_auc_score(y_test, predictions_test),
    }

    cm_test = confusion_matrix(y_test, predictions_test)
    
    # feature importance
    feature_importance = _extract_feature_importance(model, X_train.columns.values)
    
    return {
        "test_results": test_results, 
        "train_results": train_results, 
        "test_cm" : cm_test,
        "train_cm" : cm_train,
        "feature_importance": feature_importance,
        "y_pred": predictions_test,
        "model": model
    }



def run_cv(X_train, y_train, X_test, y_test, which_model):

    K_FOLD_ITERATIONS = 10

    search = GridSearchCV(
        which_model["model"], 
        which_model["params_to_tune"], 
        scoring="roc_auc",
        cv=StratifiedKFold(
            n_splits=K_FOLD_ITERATIONS, 
            shuffle=True),  
        n_jobs=-1)
    
    search.fit(X_train, y_train)
    
    model = search.best_estimator_

    # let us just print the performance on the train set
    predictions_train = model.predict(X_train)
    train_results = {
        "prec": precision_score(y_train, predictions_train),
        "rec": recall_score(y_train, predictions_train),
        "roc": roc_auc_score(y_train, predictions_train),
        "mae": mean_absolute_error(y_train, predictions_train),
    }
    cm_train = confusion_matrix(y_train, predictions_train)
    
    # in the test set
    predictions_test = model.predict(X_test)
    test_results = {
        "prec": precision_score(y_test, predictions_test),
        "rec": recall_score(y_test, predictions_test),
        "roc": roc_auc_score(y_test, predictions_test),
        "mae": mean_absolute_error(y_test, predictions_test),
    }
    cm_test = confusion_matrix(y_test, predictions_test)
    
    # feature importance
    feature_importance = _extract_feature_importance(model, X_train.columns.values)
    
    return {
        "test_results": test_results, 
        "train_results": train_results, 
        "test_cm" : cm_test,
        "train_cm" : cm_train,
        "feature_importance": feature_importance,
        "y_pred": predictions_test,
        "model": model,
        "search": search
    }


from sklearn.metrics import classification_report

def print_result(result, fi=True):
    print("Results in the training set")
    print(result["train_results"])
    print(result["train_cm"])
    print("Results in the test set")
    print(result["test_results"])
    print(result["test_cm"])
    
    print("Report:")
    print(classification_report(y_test,result["y_pred"]))
    
    if fi:
        print("Feature importances")
        pd_fi = pd.DataFrame(result["feature_importance"], columns=["feature", "importance"])
        pd_fi = pd_fi[(pd_fi.importance > 0.01)]

        p = plt.barh(pd_fi["feature"], pd_fi["importance"])
        print(p)    

In [9]:
rf_model = {
    "model": RandomForestClassifier(random_state=42),
    "params_to_tune": {
        "max_depth": [2, 3, 4, 5, 6, 7, 10, 12, None],
        "min_samples_split": [2, 3, 4, 5],
        "bootstrap": [True, False],
        "criterion": ["gini", "entropy"],
        "n_estimators": [50, 100, 150, 200]
    }
}

In [ ]:
result = run_cv(X_train, y_train, X_test, y_test, rf_model)

In [ ]:
print_result(result)

In [26]:
print(result["search"].best_score_) # roc-auc best model

0.844425087108014


In [48]:
tn, fp, fn, tp = result["test_cm"].ravel()
print(tn)
print(tp)
print(fp)
print(fn)

fpr = fp/(fp+tn)
print("false positive rate: {:.4f}".format(fpr))
fdr = fp/(fp+tp)
print("false discovery rate: {:.4f}".format(fdr))

100
6
3
12
false positive rate: 0.0291
false discovery rate: 0.3333


In [13]:
# inspired by https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65

grid_search_clf = result["model"]

y_scores = grid_search_clf.predict_proba(X_test)[:, 1]

In [14]:
from sklearn.metrics import precision_recall_curve

def adjusted_classes(y_scores, t):
    """
    This function adjusts class predictions based on the prediction threshold (t).
    Will only work for binary classification problems.
    """
    return [1 if y >= t else 0 for y in y_scores]

def precision_recall_threshold(p, r, thresholds, t=0.5, print_cm=True, plot=True):
    """
    plots the precision recall curve and shows the current value for each
    by identifying the classifier's threshold (t).
    """
    
    # generate new class predictions based on the adjusted_classes
    # function above and view the resulting confusion matrix.
    y_pred_adj = adjusted_classes(y_scores, t)
    cm = confusion_matrix(y_test, y_pred_adj)
    
    if print_cm:
        print(pd.DataFrame(cm,
           columns=['pred_neg', 'pred_pos'], 
           index=['neg', 'pos']))

    if plot:
        # plot the curve
        plt.figure(figsize=(8,8))
        plt.title("Precision and Recall curve ^ = current threshold")
        plt.step(r, p, color='b', alpha=0.2,
                 where='post')
        plt.fill_between(r, p, step='post', alpha=0.2,
                         color='b')
        plt.ylim([0.5, 1.01]);
        plt.xlim([0.5, 1.01]);
        plt.xlabel('Recall');
        plt.ylabel('Precision');

        # plot the current threshold on the line
        close_default_clf = np.argmin(np.abs(thresholds - t))
        plt.plot(r[close_default_clf], p[close_default_clf], '^', c='k',
                markersize=15)
    
    return cm

In [23]:
p, r, thresholds = precision_recall_curve(y_test, y_scores)
for i in np.arange(1, 100):
    t = i/100
    print(t)
    precision_recall_threshold(p, r, thresholds, t, True, False)

0.01
     pred_neg  pred_pos
neg        37        70
pos         0        14
0.02
     pred_neg  pred_pos
neg        47        60
pos         0        14
0.03
     pred_neg  pred_pos
neg        54        53
pos         1        13
0.04
     pred_neg  pred_pos
neg        60        47
pos         1        13
0.05
     pred_neg  pred_pos
neg        64        43
pos         1        13
0.06
     pred_neg  pred_pos
neg        68        39
pos         1        13
0.07
     pred_neg  pred_pos
neg        73        34
pos         1        13
0.08
     pred_neg  pred_pos
neg        74        33
pos         1        13
0.09
     pred_neg  pred_pos
neg        76        31
pos         2        12
0.1
     pred_neg  pred_pos
neg        76        31
pos         2        12
0.11
     pred_neg  pred_pos
neg        78        29
pos         2        12
0.12
     pred_neg  pred_pos
neg        78        29
pos         3        11
0.13
     pred_neg  pred_pos
neg        78        29
pos         3        11


In [49]:
fp_rates = np.array([])
fd_rates = np.array([])

for i in np.arange(1, 100):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    result = run_single(X_train, y_train, X_test, y_test, rf_model)
    
    tn, fp, fn, tp = result["test_cm"].ravel()
    
    fp_rates = np.append(fp_rates, (fp)/(fp+tn))    
    fd_rates = np.append(fd_rates, (fp)/(fp+tp))    

In [50]:
print ("{:.4f} (+- {:.4f})".format(fp_rates.mean(), fp_rates.std()))
print ("{:.4f} (+- {:.4f})".format(fd_rates.mean(), fd_rates.std()))

0.0298 (+- 0.0179)
0.3516 (+- 0.1608)
